More recent models, such as TSMixer, TFT and NHITS achieve better accuracy than LSTM in most settings.

In [ ]:
!pip install neuralforecast ray[tune] pytorch-lightning utilsforecast matplotlib pandas mlforecast window_ops torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set file paths
csv_path = '/content/drive/MyDrive/airkaz/selected_sensors2_cleaned.csv'
pipeline_path = '/content/drive/MyDrive/airkaz/MLForecastPipeline.py'

# Import pipeline module
import sys
sys.path.append('/content/drive/MyDrive/airkaz/')
from MLForecastPipeline import *

# Load CSV
import pandas as pd
selected_sensors_df = pd.read_csv(csv_path, index_col=0)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

class AttentiveDilatedRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size, dilation=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dilation = dilation

        self.rnn_cell = nn.GRUCell(input_size, hidden_size)
        self.attn = nn.Linear(hidden_size + input_size, 1)

    def forward(self, x_t, hidden_state):
        if hidden_state is None:
            hidden_state = torch.zeros(x_t.size(0), self.hidden_size, device=x_t.device)
        h_t = self.rnn_cell(x_t, hidden_state)
        attn_input = torch.cat([x_t, h_t], dim=-1)
        alpha = torch.sigmoid(self.attn(attn_input))
        h_t_attn = alpha * h_t + (1 - alpha) * hidden_state
        return h_t_attn

class StackedADRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dilations=(2, 4, 7)):
        super().__init__()
        self.cells = nn.ModuleList([
            AttentiveDilatedRNNCell(input_size if i == 0 else hidden_size, hidden_size, d)
            for i, d in enumerate(dilations)
        ])
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = None
        for cell in self.cells:
            outputs = []
            h = None
            for t in range(seq_len):
                h = cell(x[:, t, :], h)
                outputs.append(h.unsqueeze(1))
            x = torch.cat(outputs, dim=1)
        out = self.fc(x[:, -1, :])
        return out

In [ ]:
# Load and prepare data
df = pd.read_csv('/content/drive/MyDrive/airkaz/selected_sensors2_cleaned.csv', index_col=0)

df = df.rename(columns={'full_date': 'ds', '2': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['unique_id'] = 'sensor_2'

# Normalize
scaler = StandardScaler()
df['y_scaled'] = scaler.fit_transform(df[['y']])

# Windowing for univariate
input_size = 30
horizon = 1
X, y = [], []
for i in range(len(df) - input_size - horizon):
  X.append(df['y_scaled'].values[i:i+input_size])
  y.append(df['y_scaled'].values[i+input_size + horizon - 1])

X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # [samples, time, 1]
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # [samples, 1]

# Train-test split
split = int(0.75 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Model and training setup
model = StackedADRNN(input_size=1, hidden_size=64, output_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.SmoothL1Loss()

# Training loop
train_metrics = []
test_metrics = []

for epoch in range(500):
  model.train()
  optimizer.zero_grad()
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)
  loss.backward()
  optimizer.step()
  train_metrics.append(loss.item())

  model.eval()
  with torch.no_grad():
    y_pred_test = model(X_test).squeeze().numpy()
    y_true = y_test.squeeze().numpy()
    y_pred_inv = scaler.inverse_transform(y_pred_test.reshape(-1, 1)).squeeze()
    y_true_inv = scaler.inverse_transform(y_true.reshape(-1, 1)).squeeze()
  test_metrics.append(mean_absolute_percentage_error(y_true_inv, y_pred_inv))

  if (epoch+1) % 10 == 0:
    print(f"Epoch {epoch+1}: Train Loss = {loss.item():.4f}, Test MAPE = {test_metrics[-1]:.2%}")

# Evaluation
model.eval()
with torch.no_grad():
  y_pred = model(X_test).squeeze().numpy()
  y_true = y_test.squeeze().numpy()
  y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)).squeeze()
  y_true_inv = scaler.inverse_transform(y_true.reshape(-1, 1)).squeeze()

mape = mean_absolute_percentage_error(y_true_inv, y_pred_inv)
print(f"Test MAPE: {mape:.2%}")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(y_true_inv, label='Actual')
plt.plot(y_pred_inv, label='Forecast')
plt.title(f"adRNN Forecast (MAPE: {mape:.2%})")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Epoch 10: Train Loss = 0.5100, Test MAPE = 36.41%
Epoch 20: Train Loss = 0.3719, Test MAPE = 41.62%
Epoch 30: Train Loss = 0.3355, Test MAPE = 32.42%
Epoch 40: Train Loss = 0.3228, Test MAPE = 34.51%
Epoch 50: Train Loss = 0.3200, Test MAPE = 33.41%
Epoch 60: Train Loss = 0.3182, Test MAPE = 33.10%
Epoch 70: Train Loss = 0.3157, Test MAPE = 33.42%
Epoch 80: Train Loss = 0.3136, Test MAPE = 33.05%
Epoch 90: Train Loss = 0.3117, Test MAPE = 32.71%
Epoch 100: Train Loss = 0.3095, Test MAPE = 32.48%
Epoch 110: Train Loss = 0.3073, Test MAPE = 31.95%
Epoch 120: Train Loss = 0.3046, Test MAPE = 31.84%
Epoch 130: Train Loss = 0.3019, Test MAPE = 32.90%
Epoch 140: Train Loss = 0.2983, Test MAPE = 33.03%
Epoch 150: Train Loss = 0.2953, Test MAPE = 32.24%
Epoch 160: Train Loss = 0.2914, Test MAPE = 32.09%
Epoch 170: Train Loss = 0.2872, Test MAPE = 32.72%
Epoch 180: Train Loss = 0.2820, Test MAPE = 31.83%
Epoch 190: Train Loss = 0.2907, Test MAPE = 44.28%
Epoch 200: Train Loss = 0.3038, Test MAP

KeyboardInterrupt: 